Project Name: Detection of Road Lane Lines

Name: Prathmesh Narayan Khapare

Introduction about the overall project:
Lane Line detection is a critical component for self driving cars and also for computer vision in general. This concept is used to describe the path for self-driving cars and to avoid the risk of getting in another lane.

In [1]:
# importing the packages from libraries

import cv2
import numpy as np

In [2]:
def canny(img):
    if img is None:
        cap.release()
        cv2.destroyAllWindows()
        exit()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  #converting colour video to grey
    kernel = 5   #size of kernel
    blur = cv2.GaussianBlur(gray,(kernel, kernel),0)  #to blur the video
    canny = cv2.Canny(gray, 50, 150)  #Canny edge detection
    return canny

After running this we get the grey images and it is detecting all the edges in th images(all edges like trees,sky etc.) with too much of noise. so we are only interested in lane location so we will crop that much part for our ease.

So to do that we are creating a function called region_of_interest to avoid this extra noise.

In [3]:
def region_of_interest(canny):
    height = canny.shape[0]   #obtaining height of the image in the video
    width = canny.shape[1]    #obtaining width of the image in the video
    mask = np.zeros_like(canny)  #to remove everyrhing except the lane lines on the road
    triangle = np.array([[
    (200, height),
    (800, 350),
    (1200, height),]], np.int32)  #creating triangle
    cv2.fillPoly(mask, triangle, 255)  #masking everything except the region defined by triangle
    masked_image = cv2.bitwise_and(canny, mask)  #remove everything else and keep the region defined by triangle
    return masked_image

After running above function we got images focusing only on the triangle that is our main area of interest where the lane lines are detected. After creating the triangle now we will display this lines for this we will use hough lines algorithm.This algorithm gives an approximate presence of lines in an image.In addition to hough lines we will use average lines to obtain the exact location of line.Once these lines are detected then we go and display these lines on an image.

In [4]:
def houghLines(cropped_canny):                                  #hough line algorithm
    return cv2.HoughLinesP(cropped_canny, 2, np.pi/180, 100, 
        np.array([]), minLineLength=40, maxLineGap=5)

After hough line we use the function average_slope_intercept.It takes images and try to obtain exact slope of intercept that identifies the exact location of lines.

In [5]:
def average_slope_intercept(image, lines):
    left_fit    = []
    right_fit   = []
    if lines is None:
        return None
    for line in lines:
        for x1, y1, x2, y2 in line:
            fit = np.polyfit((x1,x2), (y1,y2), 1)
            slope = fit[0]
            intercept = fit[1]
            if slope < 0: 
                left_fit.append((slope, intercept))
            else:
                right_fit.append((slope, intercept))
    left_fit_average  = np.average(left_fit, axis=0)
    right_fit_average = np.average(right_fit, axis=0)
    left_line  = make_points(image, left_fit_average)
    right_line = make_points(image, right_fit_average)
    averaged_lines = [left_line, right_line]
    return averaged_lines

Above function finds the slope of lines in an image.

the function make_points uses the slope of line and image size and it gives how small or big line should be according to image size and slope.

In [6]:
def make_points(image, line):
    slope, intercept = line
    y1 = int(image.shape[0])
    y2 = int(y1*3.0/5)      
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return [[x1, y1, x2, y2]]

In [7]:
def display_lines(img,lines):
    line_image = np.zeros_like(img)
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv2.line(line_image,(x1,y1),(x2,y2),(0,0,255),10)
    return line_image

we will be using display_line function which is a simple open cv function of drawing lines on an image

def display_lines(img,lines):

In [8]:
def addWeighted(frame, line_image):
    return cv2.addWeighted(frame, 0.8, line_image, 1, 1)

Till now we have defined all the function based on images but we will be running all this functions on our sample video as video is a series of lots of images.

In [ ]:
cap = cv2.VideoCapture("test1.mp4")
while(cap.isOpened()):
    _, frame = cap.read()
    canny_image = canny(frame)
    cropped_canny = region_of_interest(canny_image)
    

    lines = houghLines(cropped_canny)
    averaged_lines = average_slope_intercept(frame, lines)
    line_image = display_lines(frame, averaged_lines)
    combo_image = addWeighted(frame, line_image)
    cv2.imshow("result", combo_image)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

We get our output video in which we can see that the lanes are getting detected in red color. SO using OpenCV we have detected the lanes lines.